<a href="https://colab.research.google.com/github/chen2369/data-course-sample/blob/main/Collaborative%20filtering%20%E6%8E%A8%E8%96%A6%E7%B3%BB%E7%B5%B1(using_surprise).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code

## 基礎建設

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [3]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-07 13:42:05--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.1’

All_Beauty.csv.1    100%[===================>]  14.78M  10.1MB/s    in 1.5s    

2022-01-07 13:42:07 (10.1 MB/s) - ‘All_Beauty.csv.1’ saved [15499476/15499476]

--2022-01-07 13:42:07--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.1’

meta_All_Beauty.jso 100%[===================>]   9.85M  7.55MB/s    in 1.3s   

In [4]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [5]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [6]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

metadata -> metadata_clean

In [7]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [8]:
import numpy as np

def emptylist_na(cell):
    try:
        if len(cell) == 0:
            return np.nan
        else:
            return cell
    except:
        return cell
metadata_na = metadata.applymap(emptylist_na)

In [9]:
metadata_na.isna().sum().sort_values(ascending=False).to_frame(name='Number of NA').T

,category,tech2,fit,tech1,date,feature,similar_item,also_buy,also_view,price,description,imageURLHighRes,imageURL,brand,rank,details,title,main_cat,asin
Number of NA,32892,32892,32892,32882,32873,32623,31588,26295,24760,21433,17773,16351,16351,15673,377,132,1,0,0


In [10]:
print('# main_cat:', metadata['main_cat'].unique())
print('# similar_item:', metadata['similar_item'].unique())
print('# details:', metadata['details'].sample(5))

# main_cat: ['All Beauty']
# similar_item: [''
 ' class="a-bordered a-horizontal-stripes  a-spacing-extra-large a-size-base comparison_table">\n\n\n\n            \n            \n            \n            \n            \n            <tr class="comparison_table_image_row">\n                <td class="comparison_table_first_col"></td>\n\n\n                <th class="comparison_image_title_cell" role="columnheader">\n                    <div class="a-row a-spacing-top-micro">\n                        <center>\n                             <img alt="No7 Lift &amp; Luminate Triple Action Serum 50ml by Boots" src="https://images-na.ssl-images-amazon.com/images/I/31gXVsxM7FL._SL500_AC_SS350_.jpg" id="comparison_image">\n                        </center>\n                    </div>\n                    <div class="a-row a-spacing-top-small">\n                        <div id="comparison_title" class="a-section a-spacing-none">\n                            <span aria-hidden="true" class="a-size-b

In [11]:
metadata.head(1)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]


In [12]:
(metadata.asin.value_counts()!=1).sum()

404

In [13]:
metadata_clean = metadata[['asin', 'rank', 'brand', 'price', 'also_view', 'also_buy']]
metadata_clean = metadata_clean.drop_duplicates('asin')
metadata_clean.head(1)

,asin,rank,brand,price,also_view,also_buy
0,6546546450,"2,938,573 in Beauty & Personal Care (",idea village,,[],[]


In [14]:
metadata_clean['price'] = metadata_clean['price'].str.replace('$', '')
metadata_clean['price'] = pd.to_numeric(metadata_clean['price'], errors = 'coerce')

In [15]:
metadata_clean['cate'] = metadata_clean['rank'].str.split(' in ').str[1]
metadata_clean['cate'] = metadata_clean['cate'].str.replace('amp;', '')
metadata_clean['cate'] = metadata_clean['cate'].str.replace(' \(', '')
metadata_clean['cate'].unique()

array(['Beauty & Personal Care', nan, 'Grocery & Gourmet Food',
       'Health & Household', 'Sports & Outdoors', 'Toys & Games', 'Baby',
       'Clothing, Shoes & Jewelry', 'Home & Kitchen', 'Automotive',
       'Tools & Home Improvement'], dtype=object)

In [16]:
metadata_clean = metadata_clean[metadata_clean['cate'] == 'Beauty & Personal Care']

In [17]:
metadata_clean['rank'] = metadata_clean['rank'].str.split(expand = True)[0]
metadata_clean['rank'] = metadata_clean['rank'].str.replace(',', '')
metadata_clean['rank'] = pd.to_numeric(metadata_clean['rank'])

In [18]:
metadata_clean.head(1)

,asin,rank,brand,price,also_view,also_buy,cate
0,6546546450,2938573,idea village,NaN,[],[],Beauty & Personal Care


## 資料切分

In [19]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [20]:
users_training = ratings_trainings['reviewerID'].unique()

## 產生推薦

In [21]:
!pip install surprise

In [22]:
import time
import datetime
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic

def recommender(training_data, users=[], k=10, user_based=False, algo=KNNBasic, days=9999, score=0):

    training_data = (
        training_data
        .sort_values("DATE", ascending=False)
        .groupby(['reviewerID', 'asin']).head(1)
    )
    training_data = training_data[(training_data['DATE'] + datetime.timedelta(days = days)) > '2018-09-01']
    training_data = training_data[training_data['overall'] > score]

    reader = Reader(rating_scale=(0, 5))
    training_data = training_data[['reviewerID', 'asin', 'overall']]
    data = Dataset.load_from_df(training_data, reader=reader)

    sim_options = {
        'name': 'cosine',
        'user_based': user_based  # compute similarities between items
    }
    algo_impl = algo(sim_options=sim_options)
    trainset = data.build_full_trainset()
    algo_impl.fit(trainset)

    recommendation = {}
    for user in users:
        items_user_rated = set(training_data.loc[training_data['reviewerID'] == user]['asin'].to_list())
        recommend_item_list = []
        recommend_item_set = set()
        for item in items_user_rated:
            iid = algo_impl.trainset.to_inner_iid(item)
            recommend_items_iid = algo_impl.get_neighbors(iid, k)
            for sim_item_iid in recommend_items_iid:
                item_raw_id = algo_impl.trainset.to_raw_iid(sim_item_iid)
                if item_raw_id not in items_user_rated and item_raw_id not in recommend_item_set:
                    recommend_item_list.append(item_raw_id)
                    recommend_item_set.add(item_raw_id)

            if len(recommend_item_list) >= k:
                recommend_item_list = recommend_item_list[:k]
                break
        recommendation[user] = recommend_item_list

    return recommendation

## 結果評估

In [23]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

In [24]:
for s in range(6):
    ratings_by_user = recommender(ratings_trainings, users, days=30, score=s)
    print(s, evaluate(ratings_testings_by_user, ratings_by_user))

Computing the cosine similarity matrix...
Done computing similarity matrix.
0 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
1 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
2 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
3 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
4 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
5 0.0


In [26]:
for d in range(30,181,30):
    ratings_by_user = recommender(ratings_trainings, users, days=d, score=3)
    print(d, evaluate(ratings_testings_by_user, ratings_by_user))

Computing the cosine similarity matrix...
Done computing similarity matrix.
30 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
60 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
90 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
120 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
150 0.001694915254237288
Computing the cosine similarity matrix...
Done computing similarity matrix.
180 0.001694915254237288
